In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM

os.getcwd()

'/content'

## Data

In [ ]:
# read training data - It is the aircraft engine run-to-failure data.
train_df = pd.read_csv(r'./train.csv')
print('viwe the data shape', train_df.shape)
train_df.head()

viwe the data shape (19992, 46)


,id,cycle,setting1,setting2,setting3,setting4,setting5,setting6,setting7,setting8,...,s25,s26,s27,s28,s29,s30,s31,s32,s33,RUL
0,1,1,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,-0.0007,...,392,2388,100,39.06,23.4190,17.417395,-5.424438,-11.046572,33.975011,191
1,1,2,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,0.0019,...,392,2388,100,39.00,23.4236,2.692229,6.492347,-1.090195,1.763264,190
2,1,3,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,-0.0043,...,390,2388,100,38.95,23.3442,17.608505,11.462310,-35.453907,3.903670,189
3,1,4,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,0.0007,...,392,2388,100,38.88,23.3739,2.885575,1.767567,1.996286,-3.450681,188
4,1,5,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,-0.0019,...,393,2388,100,38.90,23.4044,11.038508,8.525787,-0.011158,-1.185085,187


In [ ]:
# read test data - It is the aircraft engine operating data without failure events recorded.
test_df = pd.read_csv(r'./test.csv')
print('viwe the data shape', test_df.shape)
test_df.head()

viwe the data shape (12739, 45)


,id,cycle,setting1,setting2,setting3,setting4,setting5,setting6,setting7,setting8,...,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33
0,1,1,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,0.0023,...,0.03,392,2388,100,38.86,23.3735,5.386944,23.428504,-9.560946,-4.663842
1,1,2,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,-0.0027,...,0.03,393,2388,100,39.02,23.3916,-13.673782,21.433667,18.019028,16.455364
2,1,3,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,0.0003,...,0.03,393,2388,100,39.08,23.4166,12.873506,2.818419,8.161708,14.703674
3,1,4,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,0.0042,...,0.03,391,2388,100,39.00,23.3737,-40.262316,-5.958041,-1.516838,11.854112
4,1,5,0.533153,2.285803,0,0.591651,1.272152,5.420282,3.501178,0.0014,...,0.03,390,2388,100,38.99,23.4130,-11.969663,-3.499189,-16.507371,-20.250312


## LSTM

In [ ]:
sequence_length = 20

In [ ]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]
        
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,34)]
sequence_cols = ['setting'+ str(i) for i in range(1,11)]
sequence_cols.extend(sensor_cols)

# generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

# function to generate labels
def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]] 
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

172
(18052, 20, 43)


(18052, 1)

In [ ]:
def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])

print(model.summary())

# fit the network
history = model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=2,)

# list all data in history
print(history.history.keys())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 20, 100)           57600     
                                                                 
 dropout_6 (Dropout)         (None, 20, 100)           0         
                                                                 
 lstm_7 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
 activation_3 (Activation)   (None, 1)                 0         
                                                                 
Total params: 87,851
Trainable params: 87,851
Non-trai

In [32]:
# We pick the last sequence for each id in the test data
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

y_pred_test = model.predict(seq_array_test_last)

test_set = pd.DataFrame(y_pred_test, columns=['RUL'])
test_set.index = test_set.index+1
test_set.to_csv('my_submission.csv', index_label='id')


4/4 [==============================] - 0s 4ms/step
